# Path Optimization (KOMO)



Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')

**ry-c++-log** /home/student/Desktop/KOMO/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'
** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/student/Desktop/KOMO/rai-python/tutorials'
** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/graph.cpp:initParameters:1363(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1}



** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


Let's add some objects

In [2]:
C.addObject(name='item1', parent='sink1', shape=ry.ST.ssBox, pos=[-.1, -.1, .52], size=[.1, .1, .25, .02], color=[1., 0., 0.])
C.addObject('item2', 'sink1', ry.ST.ssBox, [.1, .1, .25, .02], [1., 1., 0.], [.1, .1, .52])
C.addObject('tray', 'stove1', ry.ST.ssBox, [.2, .2, .05, .02], [0., 1., 0.], [.0, .0, .42])

compute a collision free path to touch object item1 with pr2L:

In [3]:
X0 = C.getFrameState()

obj1 = "item1";
arm = "pr2R";

komo = C.komo_path(1.,20, 10., True);

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.distance, [arm, obj1], ry.OT.eq, [1e2]);
komo.addObjective([.9,1.], ry.FS.positionDiff, ["endeffWorkspace", obj1], ry.OT.sos, [1e0]);
komo.addObjective(times=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);

komo.optimize()
komo.getReport()

** KOMO::run solver:sparse collisions:1 x-dim:500 T:20 k:2 phases:1 stepsPerPhase:20 tau:0.5  #timeSlices:22 #totalDOFs:500 #frames:3982
** optimization time:0.870327 (kin:0.049528 coll:0.390456 feat:0 newton: 0.040159) setJointStateCount:182
   sos:5.13327 ineq:14.5443 eq:0.589962


{'F_qItself/2-#46': {'order': 2.0, 'type': 'sos', 'sos': 3.9656193830111093},
 'F_AccumulatedCollisions/0-#181': {'order': 0.0,
  'type': 'eq',
  'eq': 0.04248971122673281},
 'F_qLimits2/0-#181': {'order': 0.0,
  'type': 'ineq',
  'ineq': 14.544263306657774},
 'F_PairCollision/0-pr2R-item1': {'order': 0.0,
  'type': 'eq',
  'eq': 0.013470021063851853},
 'F_PositionDiff/0-endeffWorkspace-item1': {'order': 0.0,
  'type': 'sos',
  'sos': 1.1676555129913653},
 'F_qItself/1-#58': {'order': 1.0, 'type': 'eq', 'eq': 0.5340021346053395},
 'sos': 5.133274896002474,
 'ineq': 14.544263306657774,
 'eq': 0.5899618668959242,
 'f': 0.0}

In [4]:
komo.view(False, "my solution")

0

In [27]:
komo.view_play(False, .1)

0

Best way to read out: grab a configuration into K and analyze it here:

In [19]:
C.setFrameState( komo.getFrameState(19) )
C.getJointState()

array([-3.46070243e-01,  1.23112450e+00,  1.09339417e+00,  1.00016628e-01,
        1.08968567e-02, -1.95500178e-03,  2.53346248e-01,  1.21927441e+00,
        4.04336204e-01,  2.73031621e-01,  4.12601920e-01, -1.21375791e+00,
        9.51548379e-01, -1.30835013e+00, -2.10521793e+00, -1.72767727e-05,
       -1.69771612e-04, -1.19795831e-01, -5.01954238e-01,  2.56082928e-04,
       -4.79415870e-05,  9.82390423e-02,  9.95570671e-02,  6.32261118e-03,
        2.05960834e-02])